In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime
import import_ipynb
from tensorboard.plugins.hparams import api as hp
from dataloader import get_LA_X_df, get_y_df, data_prep

2024-01-10 22:14:10.808345: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 22:14:10.839197: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 22:14:10.840005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 22:14:11.513040: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


importing Jupyter notebook from dataloader.ipynb
Tensorflow version: 2.12.0


In [3]:
rm -rf ./logs/

In [4]:
csv_filepath = "../../data/csv/EXP1&2DATA.csv"

In [5]:
X = get_LA_X_df(csv_filepath)
y = get_y_df(csv_filepath, 'LA_mm2')

In [6]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128, 256]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(['adam', 'sgd']))
HP_ACTIVATION = hp.HParam("activation", hp.Discrete(['linear', 'relu', 'sigmoid']))

METRIC_ACCURACY = 'loss'

In [7]:
with tf.summary.create_file_writer('../../logs/hyperparam_tuning/repression/LA').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_ACTIVATION],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Test Accuracy")]
    )

2024-01-10 22:14:12.726251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-10 22:14:12.805824: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
X_train, X_test, y_train, y_test = data_prep(X, y)

In [9]:
def LA_model(hparams):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_input', input_shape = (20,),),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_dense2'),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(1, name='layers_dense3')
        ])

        model.compile (
            optimizer=hparams[HP_OPTIMIZER],
            loss='mae',
        )

        model.fit (
            X_train,
            y_train,
            epochs=250,
            validation_split=0.2,
            callbacks=[
                tf.keras.callbacks.TensorBoard(log_dir=log_dir),
                hp.KerasCallback(log_dir, hparams)
            ]
        )

        accuracy = model.evaluate(X_test, y_test)
        return accuracy

        

In [10]:
log_dir = "../../logs/hyperparam_tuning/regression/LA/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
def run_model(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = LA_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [12]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for droput in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for activation in HP_ACTIVATION.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_ACTIVATION: activation,
                    HP_DROPOUT: droput,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: optimizer
                }

                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run_model(log_dir + run_name, hparams)
                session_num += 1

--- Starting trial: run-0
{'activation': 'linear', 'dropout': 0.1, 'num_units': 16, 'optimizer': 'adam'}
Epoch 1/250


11/11 [==============================] - 1s 11ms/step - loss: 769.6277 - val_loss: 717.5264
Epoch 2/250
11/11 [==============================] - 0s 3ms/step - loss: 769.5478 - val_loss: 717.4355
Epoch 3/250
11/11 [==============================] - 0s 3ms/step - loss: 769.4628 - val_loss: 717.3480
Epoch 4/250
11/11 [==============================] - 0s 3ms/step - loss: 769.3714 - val_loss: 717.2540
Epoch 5/250
11/11 [==============================] - 0s 3ms/step - loss: 769.2731 - val_loss: 717.1541
Epoch 6/250
11/11 [==============================] - 0s 3ms/step - loss: 769.1673 - val_loss: 717.0469
Epoch 7/250
11/11 [==============================] - 0s 3ms/step - loss: 769.0536 - val_loss: 716.9500
Epoch 8/250
11/11 [==============================] - 0s 3ms/step - loss: 768.9317 - val_loss: 716.8257
Epoch 9/250
11/11 [==============================] - 0s 3ms/step - loss: 768.8013 - val_loss: 716.7016
Epoch 10/250
11/11 [==============================] - 0s 3ms/step - loss: 768.6623 -